<a href="https://colab.research.google.com/github/Anusri-J/Nasa_Neo_Asteroids_Data/blob/main/Nasa_neo_tracker_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
API_KEY = "HL3w7lWjwOGPxdPpihaX2gNFhq7EeoVQoIcta3SK"
asteroids_data = []
target = 10000

import requests
from datetime import datetime

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"


response = requests.get(url)
data = response.json()
details= data['near_earth_objects']
while len(asteroids_data) < target:
      for date,ast_details in details.items():
          for ast in ast_details:
            asteroids_data.append(dict(
                id = int(ast.get('id', 0)),
                neo_ref_id = int(ast.get('neo_reference_id', 0)),
                name = ast.get('name', 'Unknown'),
                magnitude = float(ast.get('absolute_magnitude_h', 0.0)),
                dia_min = float(ast.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_min', 0.0)),
                dia_max = float(ast.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_max', 0.0)),
                pot_haz = bool(ast.get('is_potentially_hazardous_asteroid', False)),
                close_date = datetime.strptime(ast.get('close_approach_date', '2024-01-01'), "%Y-%m-%d").date(),
                velo = float(ast.get('relative_velocity', {}).get('kilometers_per_hour', 0.0)),
                astronomical = float(ast.get('miss_distance', {}).get('astronomical', 0.0)),
                dis_km = float(ast.get('miss_distance', {}).get('kilometers', 0.0)),
                dis_lunar = float(ast.get('miss_distance', {}).get('lunar', 0.0)),
                orb_body = ast.get('orbiting_body', 'Unknown')


            ))

            if len(asteroids_data) == target:
               break
          if len(asteroids_data) == target:
              break
      url = data['links']['next']


asteroids_data

[{'id': 2415949,
  'neo_ref_id': 2415949,
  'name': '415949 (2001 XY10)',
  'magnitude': 19.37,
  'dia_min': 0.3552670883,
  'dia_max': 0.7944013596,
  'pot_haz': False,
  'close_date': datetime.date(2024, 1, 1),
  'velo': 0.0,
  'astronomical': 0.0,
  'dis_km': 0.0,
  'dis_lunar': 0.0,
  'orb_body': 'Unknown'},
 {'id': 3160747,
  'neo_ref_id': 3160747,
  'name': '(2003 SR84)',
  'magnitude': 26.0,
  'dia_min': 0.0167708462,
  'dia_max': 0.0375007522,
  'pot_haz': False,
  'close_date': datetime.date(2024, 1, 1),
  'velo': 0.0,
  'astronomical': 0.0,
  'dis_km': 0.0,
  'dis_lunar': 0.0,
  'orb_body': 'Unknown'},
 {'id': 3309828,
  'neo_ref_id': 3309828,
  'name': '(2005 YQ96)',
  'magnitude': 20.62,
  'dia_min': 0.1997813652,
  'dia_max': 0.4467247133,
  'pot_haz': True,
  'close_date': datetime.date(2024, 1, 1),
  'velo': 0.0,
  'astronomical': 0.0,
  'dis_km': 0.0,
  'dis_lunar': 0.0,
  'orb_body': 'Unknown'},
 {'id': 3457842,
  'neo_ref_id': 3457842,
  'name': '(2009 HC21)',
  'magn

In [3]:
len(asteroids_data)

10000

In [4]:
pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 57.7 MB/s eta 0:00:00


In [5]:
import mysql.connector
conn = mysql.connector.connect(host='gateway01.ap-southeast-1.prod.aws.tidbcloud.com',user='3EBy9M5V1fh5CGr.root',password='B1NWDKr2YpsTNOyT',database='nasa_asteroid_data')
cursor = conn.cursor()

In [6]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS asteroids (
    id INT,
    name VARCHAR(255),
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS close_approach (
    neo_reference_id INT,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR(255)
)
""")
conn.commit()

In [7]:
insert_asteroids = """
INSERT INTO asteroids (
    id,
    name,
    absolute_magnitude_h,
    estimated_diameter_min_km,
    estimated_diameter_max_km,
    is_potentially_hazardous_asteroid
) VALUES (%s, %s, %s, %s, %s, %s)
"""

asteroids_values = [
    (
        int(ast.get('id', 0)),
        ast.get('name', 'Unknown'),
        float(ast.get('absolute_magnitude_h', 0.0)),
        float(ast.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_min', 0.0)),
        float(ast.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_max', 0.0)),
        bool(ast.get('is_potentially_hazardous_asteroid', False))
    )
    for ast in asteroids_data
]

cursor.executemany(insert_asteroids, asteroids_values)
conn.commit()

In [8]:
insert_close_approach = """
INSERT INTO close_approach (
    neo_reference_id,
    close_approach_date,
    relative_velocity_kmph,
    astronomical,
    miss_distance_km,
    miss_distance_lunar,
    orbiting_body
) VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

close_approach_values = []

for ast in asteroids_data:
    for approach in ast.get('close_approach_data', []):
        close_approach_values.append((
            int(ast.get('id', 0)),
            datetime.strptime(approach.get('close_approach_date', '2024-01-01'), "%Y-%m-%d").date(),
            float(approach.get('relative_velocity', {}).get('kilometers_per_hour', 0.0)),
            float(approach.get('miss_distance', {}).get('astronomical', 0.0)),
            float(approach.get('miss_distance', {}).get('kilometers', 0.0)),
            float(approach.get('miss_distance', {}).get('lunar', 0.0)),
            approach.get('orbiting_body', 'Unknown')
        ))

cursor.executemany(insert_close_approach, close_approach_values)
conn.commit()

In [9]:
cursor.execute("SELECT * FROM asteroids")
data = cursor.fetchall()
#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [i[0] for i in cursor.description]
import pandas as pd
# using the result fetched in cursor, create a new dataframe
new_df = pd.DataFrame(data,columns=columns)
new_df # shows all the values inserted - 7 columns

,id,name,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid
0,123,2024 AB,22.10,0.093000,0.123000,0
1,2415949,415949 (2001 XY10),19.37,0.355267,0.794401,0
2,3160747,(2003 SR84),26.00,0.016771,0.037501,0
3,3309828,(2005 YQ96),20.62,0.199781,0.446725,1
4,3457842,(2009 HC21),22.10,0.101054,0.225964,0
...,...,...,...,...,...,...
367633,54513047,(2025 AB),0.00,0.000000,0.000000,0
367634,2450293,450293 (2004 LV3),0.00,0.000000,0.000000,0
367635,3541505,(2010 PJ),0.00,0.000000,0.000000,0
367636,3759758,(2016 SU2),0.00,0.000000,0.000000,0


In [10]:
cursor.execute("SELECT * FROM close_approach")
data = cursor.fetchall()
#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [i[0] for i in cursor.description]
import pandas as pd
# using the result fetched in cursor, create a new dataframe
new_df = pd.DataFrame(data,columns=columns)
new_df # shows all the values inserted - 7 columns

,neo_reference_id,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body
0,123,2024-01-05,21450.5,0.005,134000.0,2.1,Earth
1,2415949,2024-01-01,0.0,0.000,0.0,0.0,Unknown
2,3160747,2024-01-01,0.0,0.000,0.0,0.0,Unknown
3,3309828,2024-01-01,0.0,0.000,0.0,0.0,Unknown
4,3457842,2024-01-01,0.0,0.000,0.0,0.0,Unknown
...,...,...,...,...,...,...,...
21243,3591616,2024-01-01,0.0,0.000,0.0,0.0,Unknown
21244,3608936,2024-01-01,0.0,0.000,0.0,0.0,Unknown
21245,3795154,2024-01-01,0.0,0.000,0.0,0.0,Unknown
21246,3842680,2024-01-01,0.0,0.000,0.0,0.0,Unknown


In [11]:
pip install sqlalchemy mysql-connector-python

In [12]:
from sqlalchemy import create_engine
import pandas as pd

# Create the SQLAlchemy engine
engine = create_engine(
    "mysql+mysqlconnector://3EBy9M5V1fh5CGr.root:B1NWDKr2YpsTNOyT@"
    "gateway01.ap-southeast-1.prod.aws.tidbcloud.com:4000/nasa_asteroid_data"
)

# Define reusable function
def show_query(query):
    df = pd.read_sql_query(query, engine)
    display(df)

# SQL query
query1 = '''
SELECT neo_reference_id, COUNT(*) AS approach_count
FROM close_approach
GROUP BY neo_reference_id
ORDER BY approach_count DESC
'''

# Run the query using the function
show_query(query1)

,neo_reference_id,approach_count
0,3457842,197
1,3309828,197
2,3160747,197
3,2415949,197
4,3608936,196
...,...,...
107,54418496,188
108,54212701,188
109,3843476,188
110,54419611,188


In [13]:
query2 = '''
SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY avg_velocity DESC
'''
show_query(query2)

,neo_reference_id,avg_velocity
0,123,21450.5
1,54418797,0.0
2,3724393,0.0
3,3842680,0.0
4,54419186,0.0
...,...,...
107,54420155,0.0
108,54416853,0.0
109,3759758,0.0
110,54418498,0.0


In [14]:
query3 = '''
SELECT neo_reference_id, MAX(relative_velocity_kmph) AS top_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY top_velocity DESC
LIMIT 10
'''
show_query(query3)

,neo_reference_id,top_velocity
0,123,21450.5
1,54418649,0.0
2,54419021,0.0
3,3724393,0.0
4,3102756,0.0
5,3837605,0.0
6,54419601,0.0
7,54048870,0.0
8,3608936,0.0
9,3789479,0.0


In [15]:
query4 = '''
SELECT neo_reference_id, COUNT(*) AS approach_count
FROM close_approach
GROUP BY neo_reference_id
HAVING approach_count > 3
'''
show_query(query4)

,neo_reference_id,approach_count
0,3724393,194
1,54419186,190
2,54100190,188
3,3789479,191
4,54419601,195
...,...,...
106,54419016,190
107,2591179,190
108,2668499,190
109,2415949,197


In [16]:
query5 = '''
SELECT MONTH(close_approach_date) AS month, COUNT(*) AS approach_count
FROM close_approach
GROUP BY month
ORDER BY approach_count DESC
LIMIT 1
'''
show_query(query5)

,month,approach_count
0,1,21248


In [17]:
query6 = '''
SELECT neo_reference_id, relative_velocity_kmph
FROM close_approach
ORDER BY relative_velocity_kmph DESC
LIMIT 1
'''
show_query(query6)

,neo_reference_id,relative_velocity_kmph
0,123,21450.5


In [18]:
query7 = '''
SELECT neo_reference_id, MIN(miss_distance_km) AS closest_approach
FROM close_approach
GROUP BY neo_reference_id
ORDER BY closest_approach ASC
'''
show_query(query7)

,neo_reference_id,closest_approach
0,54048870,0.0
1,54418796,0.0
2,54417653,0.0
3,54418649,0.0
4,3795154,0.0
...,...,...
107,54397463,0.0
108,2415949,0.0
109,2591179,0.0
110,3986699,0.0


In [19]:
query8 = '''
SELECT neo_reference_id, close_approach_date, miss_distance_km
FROM close_approach
ORDER BY neo_reference_id, close_approach_date
'''
show_query(query8)

,neo_reference_id,close_approach_date,miss_distance_km
0,123,2024-01-05,134000.0
1,2199003,2024-01-01,0.0
2,2199003,2024-01-01,0.0
3,2199003,2024-01-01,0.0
4,2199003,2024-01-01,0.0
...,...,...,...
21243,54514905,2024-01-01,0.0
21244,54514905,2024-01-01,0.0
21245,54514905,2024-01-01,0.0
21246,54514905,2024-01-01,0.0


In [20]:
query9 = '''
SELECT ca.neo_reference_id, ca.close_approach_date, ca.miss_distance_km
FROM close_approach ca
JOIN (
    SELECT neo_reference_id, MIN(miss_distance_km) AS min_distance
    FROM close_approach
    GROUP BY neo_reference_id
) AS sub
ON ca.neo_reference_id = sub.neo_reference_id AND ca.miss_distance_km = sub.min_distance
'''
show_query(query9)

,neo_reference_id,close_approach_date,miss_distance_km
0,3787600,2024-01-01,0.0
1,3789479,2024-01-01,0.0
2,3837605,2024-01-01,0.0
3,3986746,2024-01-01,0.0
4,54048870,2024-01-01,0.0
...,...,...,...
21243,3591616,2024-01-01,0.0
21244,3608936,2024-01-01,0.0
21245,3795154,2024-01-01,0.0
21246,3842680,2024-01-01,0.0


In [21]:
query10 = '''
SELECT neo_reference_id, relative_velocity_kmph
FROM close_approach
WHERE relative_velocity_kmph > 50000
'''
show_query(query10)

,neo_reference_id,relative_velocity_kmph


In [22]:
query11 = '''
SELECT MONTH(close_approach_date) AS month, COUNT(*) AS total_approaches
FROM close_approach
GROUP BY month
ORDER BY month
'''
show_query(query11)

,month,total_approaches
0,1,21248


In [23]:
query12 = '''
SELECT neo_reference_id, close_approach_date, miss_distance_km
FROM close_approach
ORDER BY miss_distance_km ASC
LIMIT 1
'''
show_query(query12)

,neo_reference_id,close_approach_date,miss_distance_km
0,2415949,2024-01-01,0.0


In [24]:
query13 = '''
SELECT neo_reference_id, COUNT(*) AS total_approaches
FROM close_approach
GROUP BY neo_reference_id
ORDER BY total_approaches DESC
'''
show_query(query13)

,neo_reference_id,total_approaches
0,2415949,197
1,3309828,197
2,3160747,197
3,3457842,197
4,3591616,196
...,...,...
107,3843476,188
108,54514905,188
109,2434188,188
110,54419611,188


In [25]:
query14 = '''
SELECT neo_reference_id, close_approach_date, miss_distance_lunar
FROM close_approach
WHERE miss_distance_lunar < 1
'''
show_query(query14)

,neo_reference_id,close_approach_date,miss_distance_lunar
0,2415949,2024-01-01,0.0
1,3160747,2024-01-01,0.0
2,3309828,2024-01-01,0.0
3,3457842,2024-01-01,0.0
4,3553062,2024-01-01,0.0
...,...,...,...
21242,3591616,2024-01-01,0.0
21243,3608936,2024-01-01,0.0
21244,3795154,2024-01-01,0.0
21245,3842680,2024-01-01,0.0


In [26]:
query15 = '''
SELECT neo_reference_id, close_approach_date, astronomical AS miss_distance_au
FROM close_approach
WHERE astronomical < 0.05
'''
show_query(query15)

,neo_reference_id,close_approach_date,miss_distance_au
0,123,2024-01-05,0.005
1,2415949,2024-01-01,0.000
2,3160747,2024-01-01,0.000
3,3309828,2024-01-01,0.000
4,3457842,2024-01-01,0.000
...,...,...,...
21243,3591616,2024-01-01,0.000
21244,3608936,2024-01-01,0.000
21245,3795154,2024-01-01,0.000
21246,3842680,2024-01-01,0.000


In [27]:
#Asteroid with the highest number of approaches in a single month

query16 = '''
SELECT neo_reference_id, MONTH(close_approach_date) AS month, COUNT(*) AS approach_count
FROM close_approach
GROUP BY neo_reference_id, MONTH(close_approach_date)
ORDER BY approach_count DESC
LIMIT 1
'''
show_query(query16)

,neo_reference_id,month,approach_count
0,3160747,1,197


In [28]:
#Year with the highest total asteroid approaches

query17 = '''
SELECT YEAR(close_approach_date) AS year, COUNT(*) AS total_approaches
FROM close_approach
GROUP BY year
ORDER BY total_approaches DESC
LIMIT 1
'''
show_query(query17)

,year,total_approaches
0,2024,21248


In [29]:
#All unique orbiting bodies asteroids have approached

query18 = '''
SELECT DISTINCT orbiting_body
FROM close_approach
'''
show_query(query18)

,orbiting_body
0,Unknown
1,Earth


In [30]:
#Asteroids with multiple approaches in the same week

query19 = '''
SELECT neo_reference_id, WEEK(close_approach_date) AS week_num, COUNT(*) AS approaches
FROM close_approach
GROUP BY neo_reference_id, WEEK(close_approach_date)
HAVING approaches > 1
'''
show_query(query19)

,neo_reference_id,week_num,approaches
0,2450293,0,193
1,2676480,0,190
2,54244177,0,193
3,3595775,0,191
4,54417658,0,188
...,...,...,...
106,54407115,0,190
107,3843476,0,188
108,54048870,0,191
109,54212682,0,190


In [31]:
#Top 5 asteroids with the longest span between first and last approach

query20 = '''
SELECT neo_reference_id,
       MIN(close_approach_date) AS first_approach,
       MAX(close_approach_date) AS last_approach,
       DATEDIFF(MAX(close_approach_date), MIN(close_approach_date)) AS span_days
FROM close_approach
GROUP BY neo_reference_id
ORDER BY span_days DESC
LIMIT 5
'''
show_query(query20)

,neo_reference_id,first_approach,last_approach,span_days
0,3102756,2024-01-01,2024-01-01,0
1,54419601,2024-01-01,2024-01-01,0
2,54339170,2024-01-01,2024-01-01,0
3,3457842,2024-01-01,2024-01-01,0
4,3645001,2024-01-01,2024-01-01,0


In [32]:
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
import subprocess
subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"])
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

--2025-10-30 11:50:11--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.10.0/cloudflared-linux-amd64 [following]
--2025-10-30 11:50:11--  https://github.com/cloudflare/cloudflared/releases/download/2025.10.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/3cf4ee00-005b-4d19-8b1e-15145bd129bf?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-30T12%3A42%3A26Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-

In [33]:
%%writefile nasa_project.py

import streamlit as st
import pandas as pd
from sqlalchemy import create_engine, text

# Create SQLAlchemy engine
engine = create_engine(
    "mysql+mysqlconnector://3EBy9M5V1fh5CGr.root:B1NWDKr2YpsTNOyT@"
    "gateway01.ap-southeast-1.prod.aws.tidbcloud.com:4000/nasa_asteroid_data"
)

# Helper function
def run_query(query, params=None):
    return pd.read_sql_query(text(query), engine, params=params)

# Streamlit Config
st.set_page_config(page_title="🚀 NASA Asteroid Tracker", layout="wide")
st.title("🚀 NASA Asteroid Tracker 🪐")

# Sidebar Navigation
menu = st.sidebar.radio("Asteroid Approaches", ["Filter Criteria", "Queries"])

# ---------------------------
# Queries Section
# ---------------------------
query_options = {
    "1. Count of asteroid approaches":
        '''SELECT neo_reference_id, COUNT(*) AS approach_count
           FROM close_approach GROUP BY neo_reference_id''',

    "2. Average velocity per asteroid":
        '''SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
           FROM close_approach GROUP BY neo_reference_id''',

    "3. Top 10 fastest asteroids":
        '''SELECT neo_reference_id, MAX(relative_velocity_kmph) AS top_velocity
           FROM close_approach GROUP BY neo_reference_id
           ORDER BY top_velocity DESC LIMIT 10''',

    "4. Asteroids with more than 3 approaches":
        '''SELECT neo_reference_id, COUNT(*) AS approach_count
           FROM close_approach GROUP BY neo_reference_id HAVING approach_count > 3''',

    "5. Month with most asteroid approaches":
        '''SELECT MONTH(close_approach_date) AS month, COUNT(*) AS count
           FROM close_approach GROUP BY month ORDER BY count DESC LIMIT 1''',

    "6. Asteroid with the fastest approach":
        '''SELECT neo_reference_id, relative_velocity_kmph
           FROM close_approach ORDER BY relative_velocity_kmph DESC LIMIT 1''',

    "7. Asteroids ordered by closest approach (km)":
        '''SELECT neo_reference_id, MIN(miss_distance_km) AS closest_approach
           FROM close_approach GROUP BY neo_reference_id ORDER BY closest_approach ASC''',

    "8. Closest approaches over time":
        '''SELECT neo_reference_id, close_approach_date, miss_distance_km
           FROM close_approach ORDER BY neo_reference_id, close_approach_date''',

    "9. Closest approach per asteroid":
        '''SELECT ca.neo_reference_id, ca.close_approach_date, ca.miss_distance_km
           FROM close_approach ca
           JOIN (
               SELECT neo_reference_id, MIN(miss_distance_km) AS min_distance
               FROM close_approach GROUP BY neo_reference_id
           ) AS sub
           ON ca.neo_reference_id = sub.neo_reference_id AND ca.miss_distance_km = sub.min_distance''',

    "10. Asteroids with velocity > 50,000 km/h":
        '''SELECT neo_reference_id, relative_velocity_kmph
           FROM close_approach WHERE relative_velocity_kmph > 50000''',

    "11. Count of approaches per month":
        '''SELECT MONTH(close_approach_date) AS month, COUNT(*) AS total_approaches
           FROM close_approach GROUP BY month ORDER BY month''',

    "12. Closest ever asteroid approach (km)":
        '''SELECT neo_reference_id, close_approach_date, miss_distance_km
           FROM close_approach ORDER BY miss_distance_km ASC LIMIT 1''',

    "13. Asteroids with most frequent approaches":
        '''SELECT neo_reference_id, COUNT(*) AS total_approaches
           FROM close_approach GROUP BY neo_reference_id ORDER BY total_approaches DESC''',

    "14. Asteroids closer than 1 Lunar Distance":
        '''SELECT neo_reference_id, close_approach_date, miss_distance_lunar
           FROM close_approach WHERE miss_distance_lunar < 1''',

    "15. Asteroids that came within 0.05 AU":
        '''SELECT neo_reference_id, close_approach_date, astronomical AS miss_distance_au
           FROM close_approach WHERE astronomical < 0.05''',

    "16. Asteroid with most approaches in a single month":
        '''SELECT neo_reference_id, MONTH(close_approach_date) AS month, COUNT(*) AS approach_count
           FROM close_approach GROUP BY neo_reference_id, MONTH(close_approach_date)
           ORDER BY approach_count DESC LIMIT 1''',

    "17. Year with most total asteroid approaches":
        '''SELECT YEAR(close_approach_date) AS year, COUNT(*) AS total_approaches
           FROM close_approach GROUP BY year ORDER BY total_approaches DESC LIMIT 1''',

    "18. Unique orbiting bodies encountered":
        '''SELECT DISTINCT orbiting_body FROM close_approach''',

    "19. Asteroids with multiple approaches in same week":
        '''SELECT neo_reference_id, WEEK(close_approach_date) AS week_num, COUNT(*) AS approaches
           FROM close_approach GROUP BY neo_reference_id, WEEK(close_approach_date)
           HAVING approaches > 1''',

    "20. Top 5 asteroids with longest timespan between approaches":
        '''SELECT neo_reference_id, MIN(close_approach_date) AS first_approach,
                  MAX(close_approach_date) AS last_approach,
                  DATEDIFF(MAX(close_approach_date), MIN(close_approach_date)) AS span_days
           FROM close_approach
           GROUP BY neo_reference_id
           ORDER BY span_days DESC
           LIMIT 5'''
}

if menu == "Queries":
    st.subheader("📊 Predefined SQL Queries")

    # Single column layout
    selected_query = st.selectbox("Select your query", list(query_options.keys()), index=0)

    if selected_query:
        try:
            df = run_query(query_options[selected_query])

            if not df.empty:
                st.dataframe(df, use_container_width=True)
            else:
                st.dataframe(df, use_container_width=True)  # Just show empty table

        except Exception as e:
            st.error(f"Error executing query: {str(e)}")
            with st.expander("View SQL Query"):
                st.code(query_options[selected_query], language="sql")


# ---------------------------
# Filter Criteria Section
# ---------------------------
if menu == "Filter Criteria":
    st.subheader("🔎 Filter Asteroid Approaches")

    # Create three columns for the layout
    col1, col2, col3 = st.columns([1, 1, 1])

    # First row of filters
    with col1:
        st.markdown("**Min Magnitude**")
        min_mag = st.slider(
            "Min Magnitude",
            min_value=0.0,
            max_value=35.0,
            value=13.98,
            step=0.01,
            label_visibility="collapsed"
        )

    with col2:
        st.markdown("**Relative Velocity (km/h)**")
        vel_min, vel_max = st.slider(
            "Relative Velocity (km/h)",
            min_value=0.0,
            max_value=200000.0,
            value=(1418.21, 173971.83),
            step=0.01,
            label_visibility="collapsed"
        )

    with col3:
        st.markdown("**Start Date**")
        start_date = st.date_input(
            "Start Date",
            value=pd.to_datetime("2024-01-01"),
            label_visibility="collapsed"
        )

    # Second row of filters
    with col1:
        st.markdown("**Min Estimated Diameter (km)**")
        min_dia = st.slider(
            "Min Estimated Diameter (km)",
            min_value=0.0,
            max_value=20.0,
            value=0.0,
            step=0.1,
            label_visibility="collapsed"
        )

    with col2:
        st.markdown("**Astronomical Units (AU)**")
        au_val = st.slider(
            "Astronomical Units (AU)",
            min_value=0.0,
            max_value=1.0,
            value=0.05,
            step=0.001,
            label_visibility="collapsed"
        )

    with col3:
        st.markdown("**End Date**")
        end_date = st.date_input(
            "End Date",
            value=pd.to_datetime("2025-01-01"),
            label_visibility="collapsed"
            )

    # Third row of filters
    with col1:
        st.markdown("**Max Estimated Diameter (km)**")
        max_dia = st.slider(
            "Max Estimated Diameter (km)",
            min_value=0.0,
            max_value=20.0,
            value=10.0,
            step=0.1,
            label_visibility="collapsed"
        )

    with col2:
        st.markdown("**Lunar Distance (LD)**")
        ld_val = st.slider(
            "Lunar Distance (LD)",
            min_value=0.0,
            max_value=10.0,
            value=1.0,
            step=0.1,
            label_visibility="collapsed"
        )

    with col3:
        st.markdown("**Only Show Potentially Hazardous**")
        hazardous = st.selectbox(
            "Only Show Potentially Hazardous",
            options=["All", "Yes", "No"],
            label_visibility="collapsed"
        )

    # Filter button (centered)
    col_btn1, col_btn2, col_btn3 = st.columns([1, 2, 1])
    with col_btn1:
        filter_button = st.button("Filter", use_container_width=True)

    if filter_button:
        query = '''
        SELECT a.id, a.name, a.absolute_magnitude_h,
               a.estimated_diameter_min_km, a.estimated_diameter_max_km,
               a.is_potentially_hazardous_asteroid,
               ca.close_approach_date, ca.relative_velocity_kmph,
               ca.astronomical, ca.miss_distance_km, ca.miss_distance_lunar, ca.orbiting_body
        FROM asteroids a
        JOIN close_approach ca ON a.id = ca.neo_reference_id
        WHERE ca.close_approach_date BETWEEN :start AND :end
          AND a.absolute_magnitude_h BETWEEN :min_mag AND :max_mag
          AND a.estimated_diameter_min_km >= :min_dia
          AND a.estimated_diameter_max_km <= :max_dia
          AND ca.relative_velocity_kmph BETWEEN :vel_min AND :vel_max
          AND ca.astronomical <= :au_val
          AND ca.miss_distance_lunar <= :ld_val
        '''
        params = {
            "start": start_date, "end": end_date,
            "min_mag": min_mag, "max_mag": 35.0,
            "min_dia": min_dia, "max_dia": max_dia,
            "vel_min": vel_min, "vel_max": vel_max,
            "au_val": au_val, "ld_val": ld_val
        }
        if hazardous == "Yes":
            query += " AND a.is_potentially_hazardous_asteroid = TRUE"
        elif hazardous == "No":
            query += " AND a.is_potentially_hazardous_asteroid = FALSE"

        try:
            df = run_query(query, params)
            st.dataframe(df, use_container_width=True)
        except Exception as e:
            st.error(f"Error executing query: {str(e)}")

# Add some custom CSS for better styling
st.markdown("""
<style>
    .stSelectbox [data-testid='stMarkdownContainer'] {
        font-size: 16px;
    }
    .stButton button {
        width: 100%;
        background-color: #4F8BF9;
        color: white;
        margin-top: 20px;
        height: 50px;
        font-size: 18px;
        font-weight: bold;
    }
    .stButton button:hover {
        background-color: #3a7de8;
    }
    div[data-testid="stSlider"] > div {
        padding: 10px 0;
    }
    div[data-testid="stDateInput"] > div {
        padding: 10px 0;
    }
    div[data-testid="stSelectbox"] > div {
        padding: 10px 0;
    }
</style>
""", unsafe_allow_html=True)

Writing nasa_project.py


In [34]:
!streamlit run /content/nasa_project.py &>/content/logs.txt &

In [35]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://wisdom-arrived-middle-acre.trycloudflare.com
